# Event Bank

The EventBank class is used to interact with a local directory of event files. It's `get_events` method is compatible with the `get_events` method of the FDSN client in obspy. Additionally it has several useful features.


## Quickstart

In [1]:
import obspy
import obsplus

# Copy the Crandall dataset to a temporary directory
crandall = obsplus.copy_dataset('crandall')

# path to directory where events are stored
event_path = crandall.event_path

# Init the event bank
bank = obsplus.EventBank(event_path)

# ensure index is up-to-date
bank.update_index() 

### Accessing the index 
The index can be accessed directly to get a summary of the events contained in the archive. Depending on the task, it can be more natural to work with the index dataframe than the obspy catalog objects directly.

In [2]:
index = bank.read_index()
index

,horizontal_uncertainty,latitude,longitude,updated,event_id,p_pick_count,standard_error,depth,azimuthal_gap,magnitude,...,time,vertical_uncertainty,author,station_count,moment_magnitude,s_pick_count,s_phase_count,version,agency_id,creation_time
0,NaN,39.4625,-111.2152,1970-01-01 00:00:01.570559852,smi:local/248887,9.0,0.5716,4160.0,NaN,1.45,...,2007-08-07 03:44:18.470,NaN,DC,15.0,NaN,2.0,0.0,,NIOSH,2018-10-10 21:10:27.576204
1,NaN,39.4648,-111.2255,1970-01-01 00:00:01.570559852,smi:local/248882,19.0,0.9935,1790.0,NaN,2.08,...,2007-08-07 02:05:04.490,NaN,DC,35.0,NaN,11.0,0.0,,NIOSH,2018-10-10 21:15:19.190404
2,NaN,39.4632,-111.2230,1970-01-01 00:00:01.570559852,smi:local/248883,10.0,0.8834,4180.0,NaN,1.26,...,2007-08-07 02:14:24.080,NaN,DC,14.0,NaN,7.0,0.0,,NIOSH,2018-10-10 21:10:26.864045
3,NaN,39.4627,-111.2200,1970-01-01 00:00:01.570559852,smi:local/248925,15.0,0.5704,4620.0,NaN,1.65,...,2007-08-07 21:42:51.130,NaN,DC,19.0,NaN,7.0,0.0,,NIOSH,2018-10-11 22:08:54.236916
4,NaN,39.4605,-111.2242,1970-01-01 00:00:01.570559852,smi:local/248891,14.0,0.9901,3240.0,NaN,2.24,...,2007-08-07 07:13:05.760,NaN,,35.0,NaN,12.0,0.0,,,NaT
5,NaN,39.4635,-111.2277,1970-01-01 00:00:01.570559852,smi:local/248839,135.0,1.7356,410.0,NaN,4.37,...,2007-08-06 08:48:40.010,NaN,DC,134.0,NaN,6.0,0.0,,NIOSH,2018-10-10 20:33:13.618111
6,NaN,39.4615,-111.2317,1970-01-01 00:00:01.570559852,smi:local/248843,16.0,0.8237,2050.0,NaN,1.57,...,2007-08-06 10:47:25.600,NaN,DC,29.0,NaN,13.0,0.0,,NIOSH,2018-10-10 20:33:27.110914
7,NaN,39.4617,-111.2378,1970-01-01 00:00:01.570559852,smi:local/248828,10.0,0.8936,6570.0,NaN,1.78,...,2007-08-06 01:44:48.810,NaN,DC,23.0,NaN,6.0,0.0,,NIOSH,2018-10-10 20:26:49.642650


The index contains the following columns:

In [3]:
print(index.columns)

Index(['horizontal_uncertainty', 'latitude', 'longitude', 'updated',
       'event_id', 'p_pick_count', 'standard_error', 'depth', 'azimuthal_gap',
       'magnitude', 'magnitude_type', 'duration_magnitude', 'p_phase_count',
       'local_magnitude', 'path', 'event_description',
       'associated_phase_count', 'used_phase_count', 'time',
       'vertical_uncertainty', 'author', 'station_count', 'moment_magnitude',
       's_pick_count', 's_phase_count', 'version', 'agency_id',
       'creation_time'],
      dtype='object')


### Get events
The `EventBank` can, of course, be used to get obspy event objects based on query parameters.

In [4]:
catalog = bank.get_events(minmagnitude=2)

In [5]:
print(catalog)

3 Event(s) in Catalog:
2007-08-07T02:05:04.490000Z | +39.465, -111.225 | 2.44 ml
2007-08-07T07:13:05.760000Z | +39.461, -111.224 | 2.55 ml
2007-08-06T08:48:40.010000Z | +39.464, -111.228 | 4.2 mb


## Put events
Events can also be saved to disk using the `put_events` method. If an event with the same resource_id already exists in the bank it will be overwritten.

In [6]:
print(f'The bank has {len(bank.read_index())} events before put_events call.')

The bank has 8 events before put_events call.


In [7]:
bank.put_events(obspy.read_events())
print(f'The bank has {len(bank.read_index())} events after the put_events call.')

The bank has 11 events after the put_events call.


## Organizing event directories
`EventBank` can also be used to (re)organize event directories. For example, currently the events are saved in the following strucutre: `{year/month/day/year-month-dayThour-minute-second-short_id.xml}` (where "short_id" means the last 5 characters of the event id}. If we want to reorganize it to be `{year/month/short_id.xml}` it would be done like so:

In [8]:
from pathlib import Path

import tempfile

temp_dir = Path(tempfile.mkdtemp())

kwargs = dict(
    path_structure="{year}/{month}",
    name_structure="{event_id_short}",
)

In [9]:
print(bank.get_events())

11 Event(s) in Catalog:
2007-08-07T03:44:18.470000Z | +39.462, -111.215 | 1.68 ml
2007-08-07T02:05:04.490000Z | +39.465, -111.225 | 2.44 ml
...
2012-04-04T14:18:37.000000Z | +39.342,  +41.044 | 4.3 ML | manual
2012-04-04T14:08:46.000000Z | +38.017,  +37.736 | 3.0 ML | manual
To see all events call 'print(CatalogObject.__str__(print_all=True))'


In [10]:
bank2 = obsplus.EventBank(temp_dir, **kwargs)
bank2.put_events(bank)
print(bank2.read_index()['path'])

0     /2007/08/48887.xml
1     /2007/08/48882.xml
2     /2007/08/48883.xml
3     /2007/08/48925.xml
4     /2007/08/48891.xml
5     /2007/08/48839.xml
6     /2007/08/48843.xml
7     /2007/08/48828.xml
8     /2012/04/00041.xml
9     /2012/04/00038.xml
10    /2012/04/00039.xml
Name: path, dtype: object


## Notes
Unlike the [WaveBank](wavebank.pynb), which uses HDF5 to index waveforms, EventBank uses a [SQLite](https://www.sqlite.org/index.html) which is more suitable for frequent updates.